In [1]:
from evopt import EvolutionaryOptimizer
import numpy as np
import pandas as pd
import time
import random
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import cross_validate, train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import RobustScaler
import warnings
warnings.filterwarnings('ignore')
# Cargar dataset
df = pd.read_csv('california.csv')

# CASO DIABETES
#X = df.drop('target', axis=1).values
#y = df['target'].values

# CASO CALIFORNIA (descomentar si se usa otro dataset)
X = df.drop('MedHouseVal', axis=1).values
y = df['MedHouseVal'].values

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42
)

print(f"\n{'='*70}")
print(f"EVALUACIÓN DEL SISTEMA")
print(f"{'='*70}")
print(f"Dataset: {X.shape[0]} instancias, {X.shape[1]} features")
print(f"Train: {X_train.shape[0]} | Test: {X_test.shape[0]}")
print(f"{'='*70}\n")

# ========================================================================
# BASELINE: Modelo sin optimización
# ========================================================================
print(f"\n{'='*70}")
print(f"BASELINE (Sin Optimización)")
print(f"{'='*70}")

scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

baseline = Ridge(alpha=1.0, random_state=42)
baseline.fit(X_train, y_train)
baseline_preds = baseline.predict(X_test)

baseline_mae = mean_absolute_error(y_test, baseline_preds)
baseline_mse = mean_squared_error(y_test, baseline_preds)

print(f"MAE: {baseline_mae:.4f}")
print(f"MSE: {baseline_mse:.4f}")
print(f"Features utilizadas: {X_train.shape[1]}")

# ========================================================================
# OPTIMIZACIÓN CON PROGRAMACIÓN GENÉTICA + FEATURE SELECTION
# ========================================================================
print(f"\n{'='*70}")
print(f"OPTIMIZACIÓN EVOLUTIVA")
print(f"{'='*70}")

# Crear optimizador
gp_optimizer = EvolutionaryOptimizer(
maxtime=1200,  # 20 minutos (ajusta según necesites)
)

# Entrenar el optimizador (aprende transformaciones)
gp_optimizer.fit(X_train, y_train)

# Transformar los datos (aplicar las transformaciones aprendidas)
X_train_optimized = gp_optimizer.transform(X_train)
X_test_optimized = gp_optimizer.transform(X_test)

if gp_optimizer.feature_selection_ is not None:
    n_selected = np.sum(gp_optimizer.feature_selection_)
    n_total = len(gp_optimizer.feature_selection_)
    print(f"Features seleccionadas: {n_selected}/{n_total}")
    
    # Mostrar cuáles features se seleccionaron
    print(f"\nFeatures seleccionadas:")
    selected_indices = np.where(gp_optimizer.feature_selection_)[0]
    for idx in selected_indices:
        if idx < X.shape[1]:
            print(f"  X{idx} (original)")
        else:
            tree_idx = idx - X.shape[1]
            if tree_idx < len(gp_optimizer.best_trees_):
                print(f"  {gp_optimizer.best_trees_[tree_idx].to_string()} (generada)")


EVALUACIÓN DEL SISTEMA
Dataset: 20640 instancias, 8 features
Train: 16512 | Test: 4128


BASELINE (Sin Optimización)
MAE: 0.5332
MSE: 0.5558
Features utilizadas: 8

OPTIMIZACIÓN EVOLUTIVA

PROGRAMACIÓN GENÉTICA
Población: 100 | Features a crear: 4
Profundidad máxima: 5
Modelo de evaluación: RIDGE
Tiempo asignado GP: 14.0min (840.0s)
Tiempo asignado FS: 6.0min (360.0s)

Gen 1 - MEJORA! Val: 0.4510 | Train: 0.4550
Gen 2 - MEJORA! Val: 0.4352 | Train: 0.4448
Gen 3 - MEJORA! Val: 0.4323 | Train: 0.4429
Gen 4 - MEJORA! Val: 0.4249 | Train: 0.4377
Gen 5 - MEJORA! Val: 0.4238 | Train: 0.4360
Gen 6 - MEJORA! Val: 0.4228 | Train: 0.4350
Gen 7 - MEJORA! Val: 0.4198 | Train: 0.4307
Gen 8 - MEJORA! Val: 0.4188 | Train: 0.4297
Gen 9 - MEJORA! Val: 0.4187 | Train: 0.4296
Gen 11 - MEJORA! Val: 0.4140 | Train: 0.4260
Gen 13 - MEJORA! Val: 0.4112 | Train: 0.4212
Gen 15 - MEJORA! Val: 0.4102 | Train: 0.4202
Gen 50 | Val: 0.4102 | Train: 0.4202 | Tiempo GP: 12.4min | Early stop: 35

Programación Genétic

In [8]:
# Importar las funciones y librerías adicionales
from sklearn.ensemble import AdaBoostRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.linear_model import Lasso, ElasticNet, BayesianRidge, HuberRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.base import clone
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge
import pandas as pd

# Modelos adicionales para probar
additional_models = {
    'Ridge': Ridge(alpha=1.0, random_state=42),
    'LinearRegression': LinearRegression(),
    'RandomForest': RandomForestRegressor(n_jobs=-1, random_state=42),
    'SVR': SVR(),
    'XGBoost': XGBRegressor(n_jobs=-1, random_state=42, verbosity=0),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'Lasso': Lasso(alpha=1.0, random_state=42),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42),
    'BayesianRidge': BayesianRidge(),
    'HuberRegressor': HuberRegressor(),
    'KNeighbors': KNeighborsRegressor(n_neighbors=5),
    'DecisionTree': DecisionTreeRegressor(random_state=42),
    'ExtraTrees': ExtraTreesRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'AdaBoost': AdaBoostRegressor(n_estimators=100, random_state=42),
    'Bagging': BaggingRegressor(n_estimators=100, random_state=42, n_jobs=-1),
    'MLP': MLPRegressor(hidden_layer_sizes=(100,), max_iter=500, random_state=42),
    'KernelRidge': KernelRidge(alpha=1.0)
}

print(f"\n{'='*80}")
print("PROBANDO MODELOS ADICIONALES")
print(f"{'='*80}")

results = []

for name, model in additional_models.items():
    try:
        print(f"\nProbando {name}...")
        
        # Baseline
        model_baseline = clone(model)
        model_baseline.fit(X_train, y_train)
        baseline_preds = model_baseline.predict(X_test)
        
        baseline_mae = mean_absolute_error(y_test, baseline_preds)
        baseline_mse = mean_squared_error(y_test, baseline_preds)
        
        # Con optimización
        model_optimized = clone(model)
        model_optimized.fit(X_train_optimized, y_train)
        optimized_preds = model_optimized.predict(X_test_optimized)
        
        optimized_mae = mean_absolute_error(y_test, optimized_preds)
        optimized_mse = mean_squared_error(y_test, optimized_preds)
        
        # Mejoras
        mae_improvement = ((baseline_mae - optimized_mae) / baseline_mae * 100)
        mse_improvement = ((baseline_mse - optimized_mse) / baseline_mse * 100)
        
        results.append({
            'Modelo': name,
            'MAE_Base': baseline_mae,
            'MAE_Opt': optimized_mae,
            'Mejora_MAE': mae_improvement,
            'MSE_Base': baseline_mse,
            'MSE_Opt': optimized_mse,
            'Mejora_MSE': mse_improvement
        })
        
        print(f"  MAE: {baseline_mae:.4f} → {optimized_mae:.4f} ({mae_improvement:+.2f}%)")
        print(f"  MSE: {baseline_mse:.4f} → {optimized_mse:.4f} ({mse_improvement:+.2f}%)")
        
    except Exception as e:
        print(f"  Error: {e}")
        continue

# Mostrar resumen
print(f"\n{'='*100}")
print("RESUMEN COMPLETO - TODOS LOS MODELOS")
print(f"{'='*100}")

df_results = pd.DataFrame(results)
df_sorted = df_results.sort_values('Mejora_MSE', ascending=False)

print(f"{'Modelo':<15} {'MAE Base':<10} {'MAE Opt':<10} {'Mejora MAE':<12} {'MSE Base':<10} {'MSE Opt':<10} {'Mejora MSE':<12}")
print("-" * 100)

for _, row in df_sorted.iterrows():
    print(f"{row['Modelo']:<15} {row['MAE_Base']:<10.4f} {row['MAE_Opt']:<10.4f} "
          f"{row['Mejora_MAE']:+<12.2f}% {row['MSE_Base']:<10.4f} {row['MSE_Opt']:<10.4f} "
          f"{row['Mejora_MSE']:+<12.2f}%")

# Estadísticas finales
print(f"\n{'='*60}")
print("ESTADÍSTICAS GENERALES")
print(f"{'='*60}")
print(f"Modelos que mejoraron MAE: {len(df_sorted[df_sorted['Mejora_MAE'] > 0])}/{len(df_sorted)}")
print(f"Modelos que mejoraron MSE: {len(df_sorted[df_sorted['Mejora_MSE'] > 0])}/{len(df_sorted)}")
print(f"Mejor mejora MAE: {df_sorted['Mejora_MAE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MAE'].idxmax(), 'Modelo']})")
print(f"Mejor mejora MSE: {df_sorted['Mejora_MSE'].max():.2f}% ({df_sorted.loc[df_sorted['Mejora_MSE'].idxmax(), 'Modelo']})")
print(f"Mejora promedio MAE: {df_sorted['Mejora_MAE'].mean():.2f}%")
print(f"Mejora promedio MSE: {df_sorted['Mejora_MSE'].mean():.2f}%")


PROBANDO MODELOS ADICIONALES

Probando Ridge...
  MAE: 0.5332 → 0.4853 (+8.98%)
  MSE: 0.5558 → 0.4419 (+20.49%)

Probando LinearRegression...
  MAE: 0.5332 → 0.4854 (+8.97%)
  MSE: 0.5559 → 0.4419 (+20.51%)

Probando RandomForest...
  MAE: 0.3275 → 0.3265 (+0.31%)
  MSE: 0.2554 → 0.2617 (-2.48%)

Probando SVR...
  MAE: 0.8600 → 0.4236 (+50.74%)
  MSE: 1.3320 → 0.3903 (+70.70%)

Probando XGBoost...
  MAE: 0.3096 → 0.3194 (-3.17%)
  MSE: 0.2226 → 0.2347 (-5.46%)

Probando GradientBoosting...
  MAE: 0.3716 → 0.3669 (+1.27%)
  MSE: 0.2940 → 0.2939 (+0.04%)

Probando Lasso...
  MAE: 0.7616 → 0.9061 (-18.97%)
  MSE: 0.9380 → 1.3107 (-39.73%)

Probando ElasticNet...
  MAE: 0.6763 → 0.8409 (-24.35%)
  MSE: 0.7646 → 1.1320 (-48.06%)

Probando BayesianRidge...
  MAE: 0.5332 → 0.4854 (+8.98%)
  MSE: 0.5556 → 0.4419 (+20.46%)

Probando HuberRegressor...
  MAE: 0.5876 → 0.4811 (+18.12%)
  MSE: 0.7119 → 0.4511 (+36.63%)

Probando KNeighbors...
  MAE: 0.8128 → 0.4204 (+48.28%)
  MSE: 1.1187 → 0.399